In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pickle

from menpo.shape import PointCloud, TriMesh
from menpo.transform import PiecewiseAffine

from menpofit.deformationfield.fitter import LinearWarp
from menpofit.fittingresult import compute_error, compute_cumulative_error

# Model Analysis

Load model:

In [ ]:
model, transforms, rf, dense_shapes = pickle.load(open('/data/PhD/Models/zhouiccv2015/ear_model.pkl'))

Visualize model mean:

In [ ]:
model.mean().view_widget()

Print model and visualize its eigenspectrum:

In [ ]:
# set # of active components to toal number of components
model.n_active_components = model.n_components

model.plot_eigenvalues_cumulative_ratio()

print model

Visualize dense shape:

In [ ]:
from menpo.visualize import visualize_pointclouds

visualize_pointclouds(dense_shapes)

Obtain sparse shapes from dense ones:

In [ ]:
# set number of landmarks
n_landmarks = 55

sparse_shapes = [PointCloud(ds.points[:n_landmarks]) for ds in dense_shapes]

Visualize sparse shapes:

In [ ]:
visualize_pointclouds(sparse_shapes)

Obtain and visualize sparse landmarks on the reference frame:

In [ ]:
sparse_rf = PointCloud(rf.points[:n_landmarks])

sparse_rf.view()

Build PieceWiseAffine warp shapes:

In [ ]:
warp_shapes = [PiecewiseAffine(sparse_rf, ss).apply(rf) for ss in sparse_shapes]

(This is the triangulation being used)

In [ ]:
TriMesh(sparse_rf.points).view()

Visualize warp shapes:

In [ ]:
visualize_pointclouds(warp_shapes)

How many components capture 95% of the model's variance?

In [ ]:
model.n_active_components = 0.95

print "{} components capture 95% of the model's variance.".format(model.n_active_components)

Build LinearWarp warp transform:

In [ ]:
lw = LinearWarp(model, [], n_landmarks=n_landmarks)

Reconstruct dense shapes using the dense model (via the LinearWarp):

In [ ]:
variance_list = np.arange(0.1, 1.1, 0.1)

dense_weights = []
dense_reconstructions = []
dense_errors = []
for n in variance_list:
    lw.model.n_active_components = n
    dws = []
    drs = []
    des = []
    for ds in dense_shapes:
        # dense target
        lw.set_target(ds)
        dt = lw.target
        dw = lw.as_vector()
        de = compute_error(ds, dt)
        dws.append(dw)
        drs.append(dt)
        des.append(de)
    dense_weights.append(dws)
    dense_reconstructions.append(drs)
    dense_errors.append(des)

Reconstruct sparse shapes and warp shapes using the dense model (also via the LinearWarp):

In [ ]:
variance_list = np.arange(0.1, 1.1, 0.1)

sparse_weights = []
sparse_reconstructions = []
sparse_errors = []
warp_reconstructions = []
warp_errors = []
for n in variance_list:
    lw.model.n_active_components = n
    sws = []
    srs = []
    ses = []
    wrs = []
    wes = []
    for ss, ws in zip(sparse_shapes, warp_shapes):
        # sparse target
        lw.set_target(ss)
        st = lw.sparse_target
        sw = lw.as_vector()
        se = compute_error(ss, st)
        # piece-wise affine warp
        wr = PiecewiseAffine(sparse_rf, st).apply(rf)
        we = compute_error(ws, ww)
        # add to lists
        sws.append(sw)
        srs.append(st)
        ses.append(se)
        wrs.append(wr)
        wes.append(we)
    # add to lists
    sparse_weights.append(sws)
    sparse_reconstructions.append(srs)
    sparse_errors.append(ses)
    warp_reconstructions.append(wrs)
    warp_errors.append(wes)

Plot dense shape reconstruction error:

In [ ]:
# define sensible range for the error
x_axis = np.arange(0.0, 0.05, 0.0001)

dense_cerrors = [compute_cumulative_error(e, x_axis=x_axis) for e in dense_errors]

In [ ]:
legend =  [str(100 * j) for j in variance_list]

f = plt.figure()
plt.plot(x_axis, np.asarray(dense_cerrors).T)
plt.legend(legend, title='Model Variance')
plt.title('Dense Reconstruction Error')
f.set_size_inches((12, 6))

Plot warp shapes reconstruction error:

In [ ]:
# define sensible range for the error
x_axis = np.arange(0.0, 0.05, 0.0001)

warp_cerrors = [compute_cumulative_error(e, x_axis=x_axis) for e in warpe_errors]

In [ ]:
legend =  [str(100 * j) for j in variance_list]

f = plt.figure()
plt.plot(x_axis, np.asarray(sparse_cerrors).T)
plt.legend(legend, title='Model Variance')
plt.title('Sparse Reconstruction Error')
f.set_size_inches((12, 6))

Plot sparse shapes reconstruction error:

In [ ]:
# define sensible range for the error
x_axis = np.arange(0.0, 0.05, 0.0001)

sparse_cerrors = [compute_cumulative_error(e, x_axis=x_axis) for e in sparse_errors]

In [ ]:
legend =  [str(100 * j) for j in variance_list]

f = plt.figure()
plt.plot(x_axis, np.asarray(warp_cerrors).T)
plt.legend(legend, title='Model Variance')
plt.title('Sparse Reconstruction Error')
f.set_size_inches((12, 6))

Visualize dense reconstructions:

In [ ]:
from menpo.visualize import visualize_pointclouds

visualize_pointclouds(dense_reconstructions[-1])

Visualize warp reconstructions:

In [ ]:
from menpo.visualize import visualize_pointclouds

visualize_pointclouds(warp_reconstructions[-1])

Visualize sparse reconstructions:

In [ ]:
from menpo.visualize import visualize_pointclouds

visualize_pointclouds(sparse_reconstructions[-1])

Explore model's parameter space:

In [ ]:
c1 = 5
c2 = 6

dws = np.asarray(dense_weights[-2])
sws = np.asarray(sparse_weights[-2])

dx = dws[:, c1]
dy = dws[:, c2]

sx = sws[:, c1]
sy = sws[:, c2]


plt.figure()
plt.title('Dense Components')
plt.scatter(dx, dy)

plt.figure()
plt.title('Sparse Components')
plt.scatter(sx, sy)